# WUI & Ignitions
-  % ignitions (including tiny fires) within WUI that grew into fires bigger than say 100 acres, (by comparing with FRAP fire perimeter data). 10 > km buffers 

## Connect to Drive

In [ ]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive

drive.mount(ROOT)  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
rootPath = "/content/drive/My Drive/California FireTrends (2012-2020)"
os.chdir(rootPath)

## Import Libraries

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

In [ ]:
import pandas as pd
import time
from datetime import datetime, date, time, timedelta
import io
import seaborn as sns
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os 
import sklearn
from sklearn.metrics import mean_squared_error
from math import sqrt
#from heatmap import *
import geopandas as gpd
from shapely.geometry import Point
import folium
import rtree

In [ ]:
def lstFiles(rootPath, ext):
  '''
  retrieve file path + names based on extension
  '''
  file_list = []
  root = rootPath
  for path, subdirs, files in os.walk(root):
      for names in files: 
          if names.endswith(ext): 
              file_list.append(os.path.join(path, names))
  return(file_list)

def createFolder(rootPath, folderName): 
  '''
  Create new folder in root path 
  '''
  folderPath = os.path.join(rootPath, folderName) 
  if not os.path.exists(folderPath):
      os.makedirs(folderPath)
  return folderPath + "/"

## SHORTS Ignition w/in WUI

In [ ]:
shorts_ignition = gpd.read_file('WUI_Ignition/Shorts_2015/Shorts_Ignitions_CA.shp')

In [ ]:
shorts_ignition = shorts_ignition[['FIRE_YEAR', 'FIRE_NAME', 'FIRE_SIZE', 'DISCOVERY_', 'DISCOVER_2', 'CONT_DATE', 'STAT_CAUSE', 'STAT_CAU_1', 'geometry']]

In [ ]:
shorts_ignition = shorts_ignition[(shorts_ignition['FIRE_YEAR'].between(2010, 2015, inclusive=True)) & (shorts_ignition['FIRE_SIZE'] > 100)]

In [ ]:
len(shorts_ignition)

717

In [ ]:
wui_2010 = gpd.read_file('WUI_Ignition/CA_WUI_2010/ca_wui_10.shp')

In [ ]:
shorts_ignition = shorts_ignition.to_crs(wui_2010.crs)

In [ ]:
ignition_intersected_0km = gpd.overlay(shorts_ignition, wui_2010, how='intersection')

In [ ]:
wui_2010_10km = gpd.read_file('WUI_Ignition/CA_WUI_2010/ca_wui_10.shp')
wui_2010_10km.geometry = wui_2010_10km.geometry.buffer(5000)

In [ ]:
ignition_intersected_10km = gpd.overlay(shorts_ignition, wui_2010_10km, how='intersection')

In [ ]:
wui_class = ['Med_Dens_Intermix', 'Low_Dens_Intermix', 'Low_Dens_Interface',
       'Med_Dens_Interface', 'High_Dens_Interface', 'High_Dens_Intermix']

ignition_cause = ['Equipment Use', 'Campfire', 'Children', 'Arson', 
       'Debris Burning', 'Smoking', 'Lightning', 'Missing/Undefined', 'Miscellaneous',
       'Powerline', 'Structure', 'Railroad', 'Fireworks']

In [ ]:
len(ignition_intersected_10km[(ignition_intersected_10km['FIRE_YEAR'].between(2010, 2015, inclusive=True))])

250

In [ ]:
ignition_intersected_0km['FIRE_YEAR']

In [ ]:
len(ignition_intersected_10km[(ignition_intersected_10km['FIRE_YEAR'].between(2010, 2015, inclusive=True))])

31695

In [ ]:
# what is the extent of human influence on fire ignition? [measured by distanced from WUI areas 0, 5, 10]
# total of 14787 ignitions between 2010 to 2015 within WUI area  
# total of 717 ignitions reached over 100 acres 
# 94 ignitions reached more than 100 acres in WUI 
# 13.1% of ignitions that reached over 100 acres were within WUI areas 
# another 554 77.3% were located within 5km of WUI areas, where 453 82% are human ignited. 

# CAUSE 
# While, majority of fires that start are classified as misc. 
# 97% or 91 out of 94 were human caused, majority were equipment related, or arson.  

# LANDCHANGE
# Were there any WUI class changes? Uninhabited areas to low density etc 
# 50% Very Low Density w/ Vegetation to Low Density Intermix 
# 25% Uninhabited_Veg to low, med density 
# whereas 8.3% uninhabited _ no vegetation to low or high dnsity saw fires burn greater than 100 acres 

ignition_intersected_10_15_100 = ignition_intersected_0km[(ignition_intersected_0km['FIRE_YEAR'].between(2010, 2015, inclusive=True)) & (ignition_intersected_0km['FIRE_SIZE'] > 100)]
ignition_intersected_10_15_100_10km = ignition_intersected_10km[(ignition_intersected_10km['FIRE_YEAR'].between(2010, 2015, inclusive=True)) & (ignition_intersected_10km['FIRE_SIZE'] > 100)]

In [ ]:
ignition_intersected_10_15_100_10km

In [ ]:
ignition_intersected_10_15_100['HN_Cause'] = np.where(ignition_intersected_10_15_100['STAT_CAU_1'] == 'Lightning', 'Natural', 'Human')
ignition_intersected_10_15_100[ignition_intersected_10_15_100['HN_Cause'] == 'Human']

In [ ]:
ignition_intersected_10km['HN_Cause'] = np.where(ignition_intersected_10km['STAT_CAU_1'] == 'Lightning', 'Natural', 'Human')
ignition_intersected_10km[ignition_intersected_10km['HN_Cause'] == 'Human']

,FIRE_YEAR,FIRE_NAME,FIRE_SIZE,STAT_CAU_1,HN_Cause
0,2010,WHITES,350.0,Equipment Use,Human
2,2010,FINE,275.0,Miscellaneous,Human
3,2013,GOLD,183.0,Missing/Undefined,Human
4,2015,RED,134.0,Debris Burning,Human
16,2010,COALINGA,160.0,Equipment Use,Human
...,...,...,...,...,...
29929,2012,POWER,248.0,Miscellaneous,Human
30003,2014,WEB,378.0,Equipment Use,Human
30022,2015,TASSAJARA,1100.0,Missing/Undefined,Human
30449,2012,FLYNN 3,195.0,Miscellaneous,Human


In [ ]:
# Were there any WUI class changes? Uninhabited areas to low density etc 
ignition_intersected_10_15_100['is_equal']= (ignition_intersected_10_15_100['WUICLASS90']==ignition_intersected_10_15_100['WUICLASS10'])

# 36 instances occurred for areas that transitioned into a diff wui class from 1990 to 2010 
len(ignition_intersected_10_15_100[ignition_intersected_10_15_100['is_equal'] == False])

In [ ]:
changed_instances = ignition_intersected_10_15_100[ignition_intersected_10_15_100['is_equal'] == False]
changed_instances[['WUICLASS90', 'WUICLASS10']]

lst = list(zip(changed_instances['WUICLASS90'], changed_instances['WUICLASS10']))

from collections import Counter

Counter(lst).keys() # equals to list(set(words))
Counter(lst).values() # counts the elements' frequency

dict_values([1, 18, 4, 4, 5, 1, 2, 1])

In [ ]:
Counter(lst)

### Cause of Ignition near WUI area

In [ ]:
ignition_intersected_10_15_100 = ignition_intersected_10_15_100[['FIRE_YEAR', 'HN_Cause', 'FIRE_SIZE']]
annual_cause = ignition_intersected_10_15_100.groupby(['FIRE_YEAR', 'HN_Cause'])['FIRE_SIZE'].sum().reset_index()
annual_cause_piv = annual_cause.pivot(index='FIRE_YEAR', columns='HN_Cause', values='FIRE_SIZE').fillna(0)

In [ ]:
annual_cause_piv.columns

Index(['Arson', 'Campfire', 'Children', 'Debris Burning', 'Equipment Use',
       'Fireworks', 'Lightning', 'Miscellaneous', 'Missing/Undefined',
       'Powerline', 'Railroad', 'Smoking', 'Structure'],
      dtype='object', name='STAT_CAU_1')

In [ ]:
flatui2 = ['#bebbed', '#7e78dc', '#35ccff', '#05a167', '#66b266', '#32ecab']

plot_options = {'labelrotation':25}
sns.set_style('whitegrid', {'font.family':'serif', 'font.serif':'Times New Roman'})

ax.set_ylabel('Ratio', fontsize=12, color='tab:green')
ax = annual_cause_piv.plot(kind='bar',stacked=True, legend=True, alpha=0.5, color=flatui2)
ax.tick_params(axis='y', color='tab:green')
ax.tick_params(axis='x', **plot_options)

In [ ]:
ignition_intersected_0km[['FIRE_YEAR', 'FIRE_NAME', 'FIRE_SIZE', 'STAT_CAU_1']].drop_duplicates()

In [ ]:
ignition_intersected_10km = ignition_intersected_10km[['FIRE_YEAR', 'FIRE_NAME', 'FIRE_SIZE', 'STAT_CAU_1']].drop_duplicates()

In [ ]:
ignition_intersected_10km.groupby(['STAT_CAU_1']).count()

,FIRE_YEAR,FIRE_NAME,FIRE_SIZE,HN_Cause
STAT_CAU_1,,,,
Arson,37,36,37,37
Campfire,16,16,16,16
Children,6,6,6,6
Debris Burning,15,12,15,15
Equipment Use,83,77,83,83
Fireworks,1,1,1,1
Lightning,88,87,88,88
Miscellaneous,158,145,158,158
Missing/Undefined,116,105,116,116
